Tratar CSV

In [1]:
import datetime
import pandas as pd
import re

def timestamp_para_datahora(timestamp):
    datahora = datetime.datetime.fromtimestamp(timestamp)
    return datahora

import pandas as pd

def merge_dataframe(df, group_size=60):
    # Cria o dataframe auxiliar
    df_merged = pd.DataFrame()

    # Loop pelos grupos de tamanho "group_size"
    for i in range(0, len(df), group_size):
        # Seleciona o grupo atual
        df_group = df.iloc[i:i+group_size,:]

        # Loop pelas colunas do grupo
        for col in df_group.columns:

            # Verifica se a coluna é a datetime
            if col == 'datetime':
                # Armazena o valor da primeira linha da coluna
                col_value = df_group[col].iloc[0]

            else:
                # Armazena o valor da primeira linha da coluna
                col_value = df_group[col].iloc[0]

                # Loop pelas linhas da coluna
                for j in range(1, len(df_group)):

                    # Verifica se o valor atual é maior ou menor que o valor armazenado
                    if df_group[col].iloc[j] > col_value:
                        # Soma a diferença na variável armazenada
                        col_value += df_group[col].iloc[j] - col_value
                        
                    elif df_group[col].iloc[j] < col_value:
                        # Soma a diferença na variável armazenada
                        col_value -= col_value - df_group[col].iloc[j]

            # Insere a coluna e valor no dataframe auxiliar
            df_merged.loc[i//group_size, col] = col_value

    return df_merged



def formatar_dataframe(dataframe: pd.DataFrame) -> pd.DataFrame:
    # Converter a coluna "time" para valores numéricos e filtrar os valores nulos e não numéricos
    dataframe['time'] = pd.to_numeric(dataframe['time'], errors='coerce')
    dataframe = dataframe[~dataframe['time'].isna()]

    # Converter a coluna "time" para timestamp e criar a coluna "datetime"
    dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)

    # Apagar a coluna "time", "summary", "icon", "cloudCover"
    dataframe = dataframe.drop(columns=['time', 'summary', 'icon', 'cloudCover'])

    # Ordena alfabeticamente as colunas
    dataframe = dataframe.sort_index(axis=1)

    return merge_dataframe(dataframe)

Salvar CSV Tratado

In [2]:
import pandas as pd

filepath = '../Files/HomeCTratado.csv'
dataframe = formatar_dataframe(pd.read_csv('../Files/HomeC.csv', delimiter=',', low_memory=False))

dataframe.to_csv(filepath, index=False)

C:\Users\Alan-\AppData\Local\Temp\ipykernel_17572\3347288179.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)


Treinar modelo

In [2]:
from sklearn.ensemble import IsolationForest
import pandas as pd

#Abrir CSV ja tratado
df = pd.read_csv('../Files/HomeCTratado.csv', delimiter=',', low_memory=False)

#Colunas que não envolvem KW
exclude_columns = ['apparentTemperature','datetime','dewPoint','humidity','precipIntensity','precipProbability','pressure','temperature','visibility','windBearing','windSpeed']

#Salvar todos os dados do dataframe menos as colunas acima
X = df[[column for column in list(df.columns) if column not in exclude_columns]]

#Definir padrões pro algoritmo
isolation_forest = IsolationForest(n_estimators=100, contamination='auto')

#Treinar o algoritmo
isolation_forest.fit(X)

#Achar as anomalias
y_pred = isolation_forest.predict(X)

#Adicionar nova coluna dizendo se é ou não anomalia
df['anomaly'] = y_pred

#Dataframe somente com anomalias
anomaly = df.loc[df['anomaly'] == -1]

anomaly

,Barn [kW],Dishwasher [kW],Fridge [kW],Furnace 1 [kW],Furnace 2 [kW],Garage door [kW],Home office [kW],House overall [kW],Kitchen 12 [kW],Kitchen 14 [kW],...,humidity,precipIntensity,precipProbability,pressure,temperature,use [kW],visibility,windBearing,windSpeed,anomaly
18,0.035883,0.000050,0.593800,0.486150,0.658983,0.012333,0.041833,5.359650,0.000617,0.000650,...,0.70,0.0,0.0,1013.62,32.87,5.359650,8.06,273.0,9.14,-1
23,0.034200,1.231283,0.136100,0.492267,0.680150,0.011467,0.051483,2.908333,0.000467,0.000833,...,0.60,0.0,0.0,1014.19,30.04,2.908333,10.00,275.0,8.02,-1
24,0.034550,1.366750,0.003850,0.493400,0.071683,0.011017,0.051950,2.094050,0.000933,0.001317,...,0.60,0.0,0.0,1014.24,29.56,2.094050,10.00,273.0,8.04,-1
41,0.493183,0.000017,0.037283,0.316250,0.676550,0.012267,0.090567,2.772533,0.000850,0.000067,...,0.57,0.0,0.0,1014.28,33.22,2.772533,10.00,244.0,6.59,-1
65,0.031950,0.000033,0.118383,0.021233,0.653283,0.013133,0.679050,2.287517,0.000500,0.000017,...,0.65,0.0,0.0,1010.08,34.44,2.287517,10.00,259.0,6.55,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8295,0.029750,0.000083,0.125283,0.070383,0.664367,0.154300,0.040517,1.467100,0.000283,0.000250,...,0.50,0.0,0.0,1038.71,31.30,1.467100,10.00,152.0,4.55,-1
8322,0.031717,0.000017,0.018217,0.243667,0.606250,0.013083,0.041900,3.699633,0.000533,0.000050,...,0.55,0.0,0.0,1008.94,44.25,3.699633,10.00,308.0,14.71,-1
8335,0.029150,0.000217,0.004650,0.439733,0.063750,0.012283,0.040250,2.590267,0.000733,0.000350,...,0.68,0.0,0.0,1014.94,33.92,2.590267,9.94,291.0,9.58,-1
8375,0.032017,0.119467,0.135683,0.496117,0.063383,0.012183,0.042617,2.735383,0.000500,0.000333,...,0.81,0.0,0.0,995.90,32.63,2.735383,9.17,279.0,9.44,-1
